# Ingestion Testing

In [9]:
import datetime, json, os, rdflib, redis
ldfs_cache = redis.StrictRedis()
LUA_LOCATION = "/Users/jeremynelson/2016/linked-data-fragments/lib/"
LUA_SCRIPTS = dict()
def setup():
    for name in ["add_get_triple",
                 "triple_pattern_search",
                "get_triple"]:
        filepath = os.path.join(
            LUA_LOCATION, "{}.lua".format(name))
        print(filepath, os.path.exists(filepath))
        with open(filepath) as fo:
            lua_script = fo.read()
        sha1 = ldfs_cache.script_load(lua_script)
        LUA_SCRIPTS[name] = sha1
setup()

/Users/jeremynelson/2016/linked-data-fragments/lib/add_get_triple.lua True
/Users/jeremynelson/2016/linked-data-fragments/lib/triple_pattern_search.lua True
/Users/jeremynelson/2016/linked-data-fragments/lib/get_triple.lua True


In [10]:
def process_source(source):
    triples_processed = 0
    rec_graph = rdflib.Graph()
    rec_graph.namespace_manager.bind("", "http://dp.la/dataurl")
    rec_graph.parse(data=json.dumps(source), format='json-ld')
    for s,p,o in rec_graph:
        triples_processed += 1
        ldfs_cache.evalsha(LUA_SCRIPTS["add_get_triple"], 3, str(s), str(p), str(o))
    return triples_processed

def process_dpla_json(dpla_filepath):
    dpla_json = json.load(open(dpla_filepath, errors='ignore'))
    start = datetime.datetime.utcnow()
    total_triples = 0
    print("Started processing {} records at {}".format(len(dpla_json), start.isoformat()))
    for i,row in enumerate(dpla_json):
        total_triples += process_source(row)
        if not i%100 and i>0:
            print(".", end="")
        if not i%1000:
            print(i, end="")
        if not i%5000 and i > 0:
            print(" triples={} ".format(total_triples))
    end = datetime.datetime.utcnow()
    print("Finished processing at {}, total time {} mins. Records {} Triples {}".format(
        end.isoformat(),
        (end-start).seconds / 60.0,
        len(dpla_json),
        total_triples))

In [3]:
process_dpla_json("/Users/jeremynelson/2016/dplafest-2016/static/json/uiuc.js")

Started processing 18231 records at 2016-04-13T23:51:54.581943
0..........1000..........2000..........3000..........4000..........5000 triples=136256 
..........6000..........7000..........8000..........9000..........10000 triples=269689 
..........11000..........12000..........13000..........14000..........15000 triples=404982 
..........16000..........17000..........18000..Finished processing at 2016-04-14T00:57:45.386084, total time 65.83333333333333 mins. Records 18231 Triples 492687


In [11]:
process_dpla_json("/Users/jeremynelson/2016/dplafest-2016/static/json/getty-sample.js")
#process_dpla_json("/Users/jeremynelson/2016/dplafest-2016/static/json/smithsonian.js")

Started processing 50000 records at 2016-04-14T11:26:40.961614
0..........1000..........2000..........3000..

ConnectionResetError: [Errno 54] Connection reset by peer